## Decorator 

### Closure


In [1]:
def outer_function(): #Doesn't take any parameters
    message = "Hi"

    def inner_function():
        print(message)  #Now, the message variable wasn't created within the inner function
                        #but the inner function does have access to it. So we call this a free variable.
    return inner_function()

In [3]:
outer_function()

Hi


In [4]:
#This time, let's instead return function without executing it. 

def outer_function():
    message="Hi!"
    def inner_function():
        print(message)
    return inner_function

In [5]:
#calling the above function will return function wating to be executed. 

my_func = outer_function() 

#Now, my_func is equal to inner_function wating to be executed. 

In [6]:
my_func()
my_func()
my_func()

Hi!
Hi!
Hi!


In [7]:
def outer_function(msg): # passing in parameter
    message=msg
    def inner_function():
        print(message)
    return inner_function

In [8]:
hi_func = outer_function("Hi")
bye_func = outer_function("Bye")

In [9]:
hi_func()
bye_func()

Hi
Bye


### What is really a decorator?
Decorator is a function that:

- takes another function as an argument
- adds some kind of functionality
- returns another function.

In [10]:
def decorator_function(original_function): # passing in function as parameter
    def wrapper_function():
        return original_function()
    return wrapper_function

In [11]:
def display():
    print("display function ran")

In [12]:
decorated_display= decorator_function(display)

In [13]:
decorated_display()

display function ran


### Why do we do this?
Decorating our functions allows us to easily add functionality to our existing functions by addting that functionality inside the wrapper.

In [14]:
#example
def decorator_function(original_function): # passing in function as parameter
    def wrapper_function():
        print("wrapper executed this before {}".format(original_function.__name__))
        return original_function()
    return wrapper_function

def display():
    print("display function ran")


decorated_display = decorator_function(display)

In [16]:
decorated_display()

wrapper executed this before display
display function ran


### Wrapper

In [17]:
#Simpler way to implement the same thing
@decorator_function
def display():
    print("display function ran")

In [18]:
display() 

# This is the same as 
#
# display = decorator_fucntion(display)
#

wrapper executed this before display
display function ran


### Passing argument
When we need to be able to pass any number of positional or keyword arguments to the wrapper, we add:

*args
**kwargs

In [19]:
def decorator_function(original_function):
    def wrapper_function(*args,**kwargs):
        print("Wrapper function executed this before {}".format(original_function.__name__))
        return original_function(*args,**kwargs)
    return wrapper_function

@decorator_function
def display_info(name,age):
    print("display_info ran with arguments ({}, {})".format(name,age))

In [20]:
display_info("John",30)

Wrapper function executed this before display_info
display_info ran with arguments (John, 30)


### Classes as decorators(Although not as popular as function one)

In [21]:
#Implement the same functionality through class
class decorator_class(object):
    def __init__(self,original_function):
        self.original_function = original_function #Tie the function with the instance of this class

    def __call__(self,*args,**kwargs):
        print("Call method executed this before {}".format(self.original_function.__name__))
        return self.original_function(*args,**kwargs)

In [22]:
@decorator_class
def display_info(name,age):
    print("display_info ran with argument ({}, {})".format(name,age))

In [23]:
display_info("John",353)

Call method executed this before display_info
display_info ran with argument (John, 353)


### Pratical examples
- One of the most common use cases for decorators in python is logging
- and Timing how long functions run

In [24]:
#logging
def my_logger(orig_func):
    import logging
    #Setting up the logfile that matches the name of original function
    logging.basicConfig(filename='{}.log'.format(orig_func.__name__),level=logging.INFO)

    def wrapper(*args,**kwargs):
        logging.info(
            'Ran with args: {}, and kwargs: {}'.format(args,kwargs)
        )
        return orig_func(*args,**kwargs)
    return wrapper

In [25]:
@my_logger
def display_info(name,age):
    print("disply_info ran with arguments ({},{})".format(name,age))

display_info("John",25)

disply_info ran with arguments (John,25)


In [26]:
#Timing how long functions run
def my_timer(orig_func):
    import time
    def wrapper(*args,**kwagrs):
        t1=time.time()
        result = orig_func(*args,**kwagrs)
        t2 = time.time() -t1
        print("{} ran in: {} sec".format(orig_func.__name__,t2))
        return result
    return wrapper

In [27]:
import time
@my_timer
def display_info(name,age):
    time.sleep(1)
    print("display_info ran with arguments ({}, {})".format(name,age))

display_info("Migo",32)

display_info ran with arguments (Migo, 32)
display_info ran in: 1.0026862621307373 sec


### What if you want to apply two decorator to one function?
You can just STACK them

In [28]:
@my_logger
@my_timer
def display_info(name,age):
    time.sleep(1)
    print("display_info ran with arguments ({}, {})".format(name,age))

display_info("Laco",22)

display_info ran with arguments (Laco, 22)
display_info ran in: 1.0036699771881104 sec


### Here is a caveat
When you use two wrappers, the chances are you can't refer to original function's name because:

    outer wrapper will get the name of the function it wraps based on the return value from the inner wrapper.
    Let's take an example.

In [29]:
@my_logger
@my_timer
def display_info(name,age):
    time.sleep(1)
    print("display_info ran with arguments ({}, {})".format(name,age))

display_info("John",30)
# This is equivalent to 

display_info ran with arguments (John, 30)
display_info ran in: 1.0015292167663574 sec


In [30]:
# But the result of my_timer(display_info) is wrapper function. 
def display_info(name,age):
    time.sleep(1)
    print("display_info ran with arguments ({}, {})".format(name,age))

display_info = my_timer(display_info)
print(display_info.__name__)

wrapper


So, it makes sense logger wrapper gets the name of function as "wrapper", resulting in wrapper.log file.<br>
Then, How do we fix something like this?<br>

It's alwasy good idea to preserve the information of our original function whenever we use decorator. <br>
We can do that by using the functool module's wrap decorator.

In [31]:
from functools import wraps

#All wa have to do is decorate all of our wrappers with 'wraps' decorator
#See thw following example.

def my_logger(orig_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(orig_func.__name__),level=logging.INFO)

    #we just wrap this orignal function 
    @wraps(orig_func)
    def wrapper(*args,**kwargs):
        logging.info(
            "Ran with args: {}, nd kwargs: {}".format(args,kwargs))
        return orig_func(*args,**kwargs)
    return wrapper

def my_timer(orig_func):
    import time
    @wraps(orig_func)
    def wrapper(*args,**kwagrs):
        t1=time.time()
        result = orig_func(*args,**kwagrs)
        t2 = time.time() -t1
        print("{} ran in: {} sec".format(orig_func.__name__,t2))
        return result
    return wrapper

In [32]:
@my_logger
@my_timer
def display_info(name,age):
    time.sleep(1)
    print("display_info ran with arguments ({}, {})".format(name,age))

display_info("Hank",30)

display_info ran with arguments (Hank, 30)
display_info ran in: 1.0046803951263428 sec


In [33]:
def display_info(name,age):
    time.sleep(1)
    print("display_info ran with arguments ({}, {})".format(name,age))

display_info = my_timer(display_info)
print(display_info.__name__)

display_info


Now you can see it prints ***display_info*** instead of ***"wrapper"***<br>
The same logic goes for when you want to keep docstring of function.